# Requirements

In [6]:
import sddk
import pandas as pd
import requests
pd.options.display.max_columns = 1000 # to see all columns
import json
import re
import nltk
import warnings
from functools import partial
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import math
import sys
sys.setrecursionlimit(1000000)

In [2]:
import pyproj
from shapely.ops import transform
from shapely.geometry import Point
import geopandas as gpd
from scipy.spatial import cKDTree
#import geoplot as gplt
from pyproj import Geod
from shapely.geometry import LineString

# Import datasets

In [4]:
# read edh dataset
try: #
    EDH = pd.read_json("../../EDH_ETL_repo/data/large_data/EDH_text_cleaned_2022-11-03.json") # or replace by your own local path to the latest version of EDH
except:
    publicfolder = "b6b6afdb969d378b70929e86e58ad975"
    EDH = sddk.read_file("EDH_text_cleaned_2022-11-03.json", "df", publicfolder)

In [5]:
EDH.shape # previously we had (81476, 74)

(81883, 69)

In [6]:
# read edcs dataset
try:
    EDCS = pd.read_json("../data/large_data/EDCS_text_cleaned_2022-09-12.json") # or replace by your own local path to the latest version of EDCS
except:
    publicfolder = "1f5f56d09903fe259c0906add8b3a55e"
    EDCS = sddk.read_file("EDCS_text_cleaned_2022-09-12.json", "df", publicfolder)
    EDCS.to_json("../data/large_data/EDCS_text_cleaned_2022-09-12.json")

In [7]:
EDCS.shape

(537262, 27)

In [8]:
EDCS.head()

,EDCS-ID,publication,province,place,inscription,inscription_conservative_cleaning,inscription_interpretive_cleaning,inscr_type,status_notation,inscr_process,clean_text_conservative,clean_text_interpretive_word,status,material,partner_link,dating_from,dating_to,date_not_before,date_not_after,latitude,longitude,photo,raw_dating,language,comment,extra_text,extra_html
0,EDCS-31400030,"CIL 03, 12297",Achaia,?,Leius,Leius,Leius,{},{},{},Leius,Leius,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None
1,EDCS-24700151,"CIL 01, 02650 (p 1097) = IG-05-01, 00741 = ILL...",Achaia,Afesou,Δέκιος Λείβιος (Ζ)εῦξις // D(ecimi) Leivei D(e...,Δέκιος Λείβιος εῦξις D Leivei D Leivei salve,Δέκιος Λείβιος Ζεῦξις Decimi Leivei Decimi Lei...,tituli sepulcrales,"[praenomen et nomen, viri]",{},Δέκιος Λείβιος εῦξις D Leivei D Leivei salve,Δέκιος Λείβιος Ζεῦξις Decimi Leivei Decimi Lei...,praenomen et nomen; tituli sepulcrales; viri,lapis,http://db.edcs.eu/epigr/partner.php?s_language...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None
2,EDCS-24900077,"CIL 01, 00746 (p 944) = D 00867 = ILLRP 00374 ...",Achaia,Agia Triada / Merbaka / Midea,Q(uinto) Caecilio C(ai) f(ilio) Metel(l)o / im...,Q Caecilio C f Metelo imperatori Italici quei ...,Quinto Caecilio Cai filio Metello imperatori I...,tituli honorarii,"[officium/professio, ordo senatorius, tria nom...",{},Q Caecilio C f Metelo imperatori Italici quei ...,Quinto Caecilio Cai filio Metello imperatori I...,officium/professio; ordo senatorius; tituli ...,None,http://db.edcs.eu/epigr/partner.php?s_language...,-68.0,-68.0,-68.0,-68.0,37.638113,22.805299,http://db.edcs.eu/epigr/bilder.php?bilder.php?...,-68 to -68,None,None,None,None
3,EDCS-03700724,"ZPE-108-159 = Thesprotia 00001 = AE 1993, 0140...",Achaia,Agios Athanasios / Photike,Fortissimo et Piis/simo Caesari d(omino) n(ost...,Fortissimo et Piissimo Caesari d n Gal Val P F...,Fortissimo et Piissimo Caesari domino nostro G...,tituli honorarii,"[Augusti/Augustae, ordo equester, tria nomina]",litterae erasae,Fortissimo et Piissimo Caesari d n Gal Val P F...,Fortissimo et Piissimo Caesari domino nostro G...,Augusti/Augustae; litterae erasae; ordo eque...,None,http://db.edcs.eu/epigr/partner.php?s_language...,309.0,313.0,309.0,313.0,39.451218,20.766767,http://db.edcs.eu/epigr/bilder.php?bilder.php?...,309 to 313,None,None,None,None
4,EDCS-55701593,"AE 2009, 01286a",Achaia,Agios Donatos / Photike,Cn(aeus) Atei(us),Cn Atei,Cnaeus Ateius,tituli fabricationis,"[praenomen et nomen, viri]",sigilla impressa,Cn Atei,Cnaeus Ateius,praenomen et nomen; sigilla impressa; tituli...,opus figlinae,None,NaN,NaN,NaN,NaN,39.475976,20.506908,None,None,None,None,None,None


In [9]:
# replace empty dicts and Nones by np.non
def clean_empty(el):
    if isinstance(el,dict):
        if ~bool(el):
            el = np.nan
    elif el == None: #(el == np.nan) | (el == math.nan) | (str(el)=="NaN"):
        el = np.nan
    else:
        el = el
    return el
for col in EDCS.columns:
    clean_col = EDCS[col].apply(clean_empty)
    EDCS[col] = clean_col

for col in EDH.columns:
    clean_col = EDH[col].apply(clean_empty)
    EDH[col] = clean_col

In [10]:
EDCS.head(5)

,EDCS-ID,publication,province,place,inscription,inscription_conservative_cleaning,inscription_interpretive_cleaning,inscr_type,status_notation,inscr_process,clean_text_conservative,clean_text_interpretive_word,status,material,partner_link,dating_from,dating_to,date_not_before,date_not_after,latitude,longitude,photo,raw_dating,language,comment,extra_text,extra_html
0,EDCS-31400030,"CIL 03, 12297",Achaia,?,Leius,Leius,Leius,NaN,NaN,NaN,Leius,Leius,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EDCS-24700151,"CIL 01, 02650 (p 1097) = IG-05-01, 00741 = ILL...",Achaia,Afesou,Δέκιος Λείβιος (Ζ)εῦξις // D(ecimi) Leivei D(e...,Δέκιος Λείβιος εῦξις D Leivei D Leivei salve,Δέκιος Λείβιος Ζεῦξις Decimi Leivei Decimi Lei...,tituli sepulcrales,"[praenomen et nomen, viri]",NaN,Δέκιος Λείβιος εῦξις D Leivei D Leivei salve,Δέκιος Λείβιος Ζεῦξις Decimi Leivei Decimi Lei...,praenomen et nomen; tituli sepulcrales; viri,lapis,http://db.edcs.eu/epigr/partner.php?s_language...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EDCS-24900077,"CIL 01, 00746 (p 944) = D 00867 = ILLRP 00374 ...",Achaia,Agia Triada / Merbaka / Midea,Q(uinto) Caecilio C(ai) f(ilio) Metel(l)o / im...,Q Caecilio C f Metelo imperatori Italici quei ...,Quinto Caecilio Cai filio Metello imperatori I...,tituli honorarii,"[officium/professio, ordo senatorius, tria nom...",NaN,Q Caecilio C f Metelo imperatori Italici quei ...,Quinto Caecilio Cai filio Metello imperatori I...,officium/professio; ordo senatorius; tituli ...,NaN,http://db.edcs.eu/epigr/partner.php?s_language...,-68.0,-68.0,-68.0,-68.0,37.638113,22.805299,http://db.edcs.eu/epigr/bilder.php?bilder.php?...,-68 to -68,NaN,NaN,NaN,NaN
3,EDCS-03700724,"ZPE-108-159 = Thesprotia 00001 = AE 1993, 0140...",Achaia,Agios Athanasios / Photike,Fortissimo et Piis/simo Caesari d(omino) n(ost...,Fortissimo et Piissimo Caesari d n Gal Val P F...,Fortissimo et Piissimo Caesari domino nostro G...,tituli honorarii,"[Augusti/Augustae, ordo equester, tria nomina]",litterae erasae,Fortissimo et Piissimo Caesari d n Gal Val P F...,Fortissimo et Piissimo Caesari domino nostro G...,Augusti/Augustae; litterae erasae; ordo eque...,NaN,http://db.edcs.eu/epigr/partner.php?s_language...,309.0,313.0,309.0,313.0,39.451218,20.766767,http://db.edcs.eu/epigr/bilder.php?bilder.php?...,309 to 313,NaN,NaN,NaN,NaN
4,EDCS-55701593,"AE 2009, 01286a",Achaia,Agios Donatos / Photike,Cn(aeus) Atei(us),Cn Atei,Cnaeus Ateius,tituli fabricationis,"[praenomen et nomen, viri]",sigilla impressa,Cn Atei,Cnaeus Ateius,praenomen et nomen; sigilla impressa; tituli...,opus figlinae,NaN,NaN,NaN,NaN,NaN,39.475976,20.506908,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# length of the original dataset
print(len(EDH))
print(len(EDCS))

81883
537262


In [12]:
EDCS.rename(columns={"longitude": "Longitude", "latitude": "Latitude"}, inplace=True)

In [13]:
len(EDCS[EDCS["Longitude"].notnull()])

523136

In [14]:
EDCS_sample = EDCS.sample(1000)
EDCS_sample.head(5)

,EDCS-ID,publication,province,place,inscription,inscription_conservative_cleaning,inscription_interpretive_cleaning,inscr_type,status_notation,inscr_process,clean_text_conservative,clean_text_interpretive_word,status,material,partner_link,dating_from,dating_to,date_not_before,date_not_after,Latitude,Longitude,photo,raw_dating,language,comment,extra_text,extra_html
412529,EDCS-24100649,"CIL 06, 33879 (p 3896) = D 05310 = Gummerus-01...",Roma,Roma,Hyla medicus / factionis / venetae se vivo / [...,Hyla medicus factionis venetae se vivo I sibi ...,Hyla medicus factionis venetae se vivo I sibi ...,NaN,NaN,NaN,Hyla medicus factionis venetae se vivo I sibi ...,Hyla medicus factionis venetae se vivo I sibi ...,NaN,NaN,http://db.edcs.eu/epigr/partner.php?s_language...,1.0,50.0,1.0,50.0,41.893320,12.482932,NaN,1 to 50,NaN,NaN,NaN,NaN
458852,EDCS-37200718,"ICUR-04, 11317",Roma,Roma,Ioanne[s q(ui) vix(it)] / annu(m) un[um mens(e...,Ioanne annu un quattuor III Idus Oc,Ioannes qui vixit annum unum menses quattuor d...,"[inscriptiones christianae, tituli sepulcrales]","[nomen singulare, viri]",NaN,Ioanne annu un quattuor III Idus Oc,Ioannes qui vixit annum unum menses quattuor d...,inscriptiones christianae; nomen singulare; ...,NaN,http://db.edcs.eu/epigr/partner.php?s_language...,401.0,500.0,401.0,500.0,41.893320,12.482932,NaN,401 to 500,NaN,NaN,NaN,NaN
233802,EDCS-56800162,"CIL 02-14, 01811",Hispania citerior,Tarragona / Tarraco / Terraco,] / eum legis adlege / [,eum legis adlege,eum legis adlege,NaN,NaN,NaN,eum legis adlege,eum legis adlege,NaN,lapis,NaN,NaN,NaN,NaN,NaN,41.117236,1.254606,http://db.edcs.eu/epigr/bilder.php?bilder.php?...,NaN,NaN,NaN,NaN,NaN
342142,EDCS-13700112,"ILAlg-02-02, 05808",Numidia,Sellaoua Announa / Salaoua Announa / El Announ...,D(is) M(anibus) s(acrum) / M(arcus) Seius / Ne...,D M s M Seius Nepos v a XLI Fuficia Honorata v a,Dis Manibus sacrum Marcus Seius Nepos vixit an...,tituli sepulcrales,NaN,NaN,D M s M Seius Nepos v a XLI Fuficia Honorata v a,Dis Manibus sacrum Marcus Seius Nepos vixit an...,tituli sepulcrales,lapis,NaN,NaN,NaN,NaN,NaN,36.386768,7.250702,NaN,NaN,NaN,NaN,NaN,NaN
315763,EDCS-08800110,"IAM-02-01, 00126 = IAM-S, 00126 = MEFR-2012-22...",Mauretania Tingitana,Sidi Ali bou Jenoun / Sidi Ali bou Djenoun / B...,Imp(eratore) Caesare Vespasiano / Aug(usto) VI...,Imp Caesare Vespasiano Aug VI T Imp Aug f IIII...,Imperatore Caesare Vespasiano Augusto VI Tito ...,leges,"[ordo senatorius, tria nomina, viri]",NaN,Imp Caesare Vespasiano Aug VI T Imp Aug f IIII...,Imperatore Caesare Vespasiano Augusto VI Tito ...,leges; ordo senatorius; tria nomina; viri,lapis,http://db.edcs.eu/epigr/partner.php?s_language...,75.0,75.0,75.0,75.0,34.602296,-6.115147,http://db.edcs.eu/epigr/bilder.php?bilder.php?...,75 to 75,NaN,NaN,NaN,NaN


In [36]:
#EDCS_sample.to_json("../data/EDCS_sample.json", orient="columns")

# Get EDH-ID from EDCS "Links" column

In [15]:
len([re.search("HD\d+", str(el))[0] for el in EDCS["partner_link"].tolist() if re.search("HD\d+", str(el))])

76763

In [16]:
def get_edh_id(partner_link):
    try:
        if re.search("HD\d+", str(partner_link)):
            edh_id = re.search("HD\d+", str(partner_link))[0]
        else:
            edh_id = None
    except:
        edh_id = None
    return edh_id
EDCS["EDH-ID"] = EDCS["partner_link"].apply(get_edh_id)

In [17]:
EDCS["EDH-ID"].notnull().sum()

76763

In [18]:
# unique EDH
len(set(EDCS["EDH-ID"]))

75557

In [19]:
edh_ids = list(set([el for el in EDCS["EDH-ID"].tolist() if el != None]))
print(edh_ids[:10])

['HD053362', 'HD037718', 'HD062387', 'HD023474', 'HD047795', 'HD028803', 'HD031132', 'HD042328', 'HD002378', 'HD030065']


In [20]:
# percents covered: 
len(edh_ids) / len(EDH) * 100

92.27312140493143

In [21]:
# inscriptions which are not explicitly refered to in EDCS
EDH.rename(columns={"id":"EDH-ID"}, inplace=True)
EDH_unique = EDH[~EDH["EDH-ID"].isin(edh_ids)]
len(EDH_unique) # previously we had 8450

6834

what about these?

# Get EDH-ID from trismegistos API

EDH standardly includes references to trismegistos. The trismegistos URI api enables queries for EDCS-ID. We will use this extract EDCS-IDs of additional inscriptions in EDCS which are in EDH  

In [28]:
def get_edcs(trismegistos_uri):
    try:
        url = "https://www.trismegistos.org/dataservices/texrelations/{}".format(trismegistos_uri.rpartition("/")[2])
        resp = requests.get(url)
        edcs_ids = resp.json()[3]["EDCS"]
        return edcs_ids
    except: 
        return None

In [29]:
%%time
# takes time, so use it carefully, or use data from the previous run below
EDH_unique["edcs_ids"] = EDH_unique["trismegistos_uri"].apply(get_edcs)

CPU times: user 1min 4s, sys: 8.71 s, total: 1min 13s
Wall time: 19min 35s


In [30]:
EDH_unique

,commentary,country,depth,diplomatic_text,findspot_ancient,findspot_modern,height,EDH-ID,language,last_update,letter_size,literature,material,modern_region,not_after,not_before,responsible_individual,transcription,trismegistos_uri,type_of_inscription,type_of_monument,width,work_status,findspot,year_of_find,present_location,religion,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,people,province_label,pleiades_id,Longitude,Latitude,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,origdate_text_clean,clean_text_conservative,clean_text_interpretive_word,clean_text_interpretive_sentence,edcs_ids
305,(B): Z. 3: Synphor[us].,Italy,2 cm,D M / QVINTAE / SYNPHOR / CONSERVAE B M / ET E...,"Luceria, bei","Lucera, bei",(28) cm,HD000306,L,2015-10-05,4-2.3 cm,"AE 1983, 0240. (B) ; M. Bàlice, ASP 34, 1981, ...",Kalkstein,Foggia,151.0,200.0,Feraudi,D(is) M(anibus) / Quintae / Synphor / conserva...,https://www.trismegistos.org/text/245296,epitaph,tabula,34 cm,provisional,"Fluß Casanova, bei",1956.0,"Lucera, Mus. Civ. """"Giuseppe Fiorelli""""",NaN,NaN,"[{'type': 'province', 'text': 'Apulia et Calab...",\n Dis Manibus Quintae Synphor conservae bene ...,151 AD - 200 AD,NaN,NaN,257,tabula,60.0,Kalkstein,1000.0,92.0,epitaph,"[{'persname': {'name': {'@type': 'cognomen', '...",Apulia et Calabria (Regio II),442640,15.3359,41.5092,epitaph,Certain,28.0,34.0,2.0,Limestone,tabula,Certain,Apulia et Calabria (Regio II),Certain,Italy,Certain,Luceria,Estimated,Foggia,Certain,"Lucera, bei",Certain,"Fluß Casanova, bei",Certain,151 AD - 200 AD,D M Quintae Synphor conservae b m et Elaphe ma...,Dis Manibus Quintae Synphor conservae bene mer...,Dis Manibus Quintae Synphor conservae bene mer...,None
511,Fragment einer Vase mit appliziertem Medaillon...,France,NaN,ANDROMACHE / / PRIAMVS / / HECV / BA / / ACHIL...,Lugdunum,Lyon,NaN,HD000513,L,2018-12-03,NaN,"AE 1982, 0712, 11. ; A. Desbat, Vases à médail...",Ton,Auvergne-Rhône-Alpes,101.0,200.0,Feraudi,Andromache // Priamus // Hecu/ba // Achil/les ...,https://www.trismegistos.org/text/212619,label,instrumentum domesticum,NaN,checked with drawing,"La Solitude, Nymphaeum",1967.0,NaN,NaN,NaN,"[{'type': 'province', 'text': 'Lugdunensis'}, ...",\n Andromache Priamus Hecuba Achilles Hector a...,101 AD - 200 AD,NaN,NaN,140,instrumentum domesticum,131.0,Ton,2000.0,109.0,label,NaN,Lugdunensis,167717,4.8245,45.7574,label,Certain,NaN,NaN,NaN,Clay,instrumentum domesticum,Certain,Lugdunensis,Certain,France,Certain,Lugdunum,Certain,Auvergne-Rhône-Alpes,Certain,Lyon,Certain,"La Solitude, Nymphaeum",Certain,101 AD - 200 AD,Andromache Priamus Hecuba Achilles Hector aurum,Andromache Priamus Hecuba Achilles Hector aurum,Andromache Priamus Hecuba Achilles Hector aurum,[08600519]
544,(B): Z. 3/4: Costorius Italus d(onum) d(edit) ...,Italy,NaN,ALBVLAE ISIDI / DEANAM / C OSTORIVS ITALVS / D D,Aquae Albulae,"Tivoli, bei",27 cm,HD000547,L,2006-08-31,NaN,"AE 1983, 0166. ; Z. Mari, Tibur, Forma Italiae...",Marmor,Roma,-100.0,-1.0,Feraudi,Albulae Isidi / Deanam(!) / C(aius) Ostorius I...,https://www.trismegistos.org/text/251173,votive inscription,statue base,25 cm,provisional,Azienda agricola Massicci,1926.0,NaN,names of pagan deities,NaN,"[{'type': 'province', 'text': 'Latium et Campa...",\n Albulae Isidi Deanam sic Caius Ostorius Ita...,100 BC - 1 BC,NaN,NaN,57,statue base,48.0,Marmor,1000.0,372.0,votive inscription,NaN,Latium et Campania (Regio I),422841,12.7983,41.9636,votive inscription,Certain,27.0,25.0,NaN,Marble,stat

In [32]:
edcs_to_edh_dict = {}
for edh_id, edcs_ids in zip(EDH_unique["EDH-ID"], EDH_unique["edcs_ids"]):
    try:
        for edcs_id in edcs_ids:
            edcs_to_edh_dict["EDCS-" + edcs_id] = edh_id
    except:
        pass

In [34]:
# save for future usage
with open("../data/edcs_to_edh_dict.json", "w") as f:
    json.dump(edcs_to_edh_dict, f)

In [45]:
# upload the previously identified pairs
with open("../data/edcs_to_edh_dict.json", "r") as f:
    edcs_to_edh_dict = json.load(f)

In [46]:
len(EDCS[EDCS["EDH-ID"].notnull()])

80452

In [47]:
def get_edh_id_trismegistos(row):
    edh_id = row["EDH-ID"]
    if row["EDH-ID"] == None:
        if row["EDCS-ID"] in edcs_to_edh_dict.keys():
            edh_id = edcs_to_edh_dict[row["EDCS-ID"]]  
    return edh_id

In [48]:
EDCS["EDH-ID"] = EDCS.apply(lambda row: get_edh_id_trismegistos(row), axis=1)

In [49]:
EDCS[EDCS["EDH-ID"].notnull()].head(10)

,EDCS-ID,publication,province,place,inscription,inscription_conservative_cleaning,inscription_interpretive_cleaning,inscr_type,status_notation,inscr_process,clean_text_conservative,clean_text_interpretive_word,status,material,partner_link,dating_from,dating_to,date_not_before,date_not_after,Latitude,Longitude,photo,raw_dating,language,comment,extra_text,extra_html,EDH-ID
1,EDCS-24700151,"CIL 01, 02650 (p 1097) = IG-05-01, 00741 = ILL...",Achaia,Afesou,Δέκιος Λείβιος (Ζ)εῦξις // D(ecimi) Leivei D(e...,Δέκιος Λείβιος εῦξις D Leivei D Leivei salve,Δέκιος Λείβιος Ζεῦξις Decimi Leivei Decimi Lei...,tituli sepulcrales,"[praenomen et nomen, viri]",NaN,Δέκιος Λείβιος εῦξις D Leivei D Leivei salve,Δέκιος Λείβιος Ζεῦξις Decimi Leivei Decimi Lei...,praenomen et nomen; tituli sepulcrales; viri,lapis,http://db.edcs.eu/epigr/partner.php?s_language...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HD057726
2,EDCS-24900077,"CIL 01, 00746 (p 944) = D 00867 = ILLRP 00374 ...",Achaia,Agia Triada / Merbaka / Midea,Q(uinto) Caecilio C(ai) f(ilio) Metel(l)o / im...,Q Caecilio C f Metelo imperatori Italici quei ...,Quinto Caecilio Cai filio Metello imperatori I...,tituli honorarii,"[officium/professio, ordo senatorius, tria nom...",NaN,Q Caecilio C f Metelo imperatori Italici quei ...,Quinto Caecilio Cai filio Metello imperatori I...,officium/professio; ordo senatorius; tituli ...,NaN,http://db.edcs.eu/epigr/partner.php?s_language...,-68.0,-68.0,-68.0,-68.0,37.638113,22.805299,http://db.edcs.eu/epigr/bilder.php?bilder.php?...,-68 to -68,NaN,NaN,NaN,NaN,HD056163
3,EDCS-03700724,"ZPE-108-159 = Thesprotia 00001 = AE 1993, 0140...",Achaia,Agios Athanasios / Photike,Fortissimo et Piis/simo Caesari d(omino) n(ost...,Fortissimo et Piissimo Caesari d n Gal Val P F...,Fortissimo et Piissimo Caesari domino nostro G...,tituli honorarii,"[Augusti/Augustae, ordo equester, tria nomina]",litterae erasae,Fortissimo et Piissimo Caesari d n Gal Val P F...,Fortissimo et Piissimo Caesari domino nostro G...,Augusti/Augustae; litterae erasae; ordo eque...,NaN,http://db.edcs.eu/epigr/partner.php?s_language...,309.0,313.0,309.0,313.0,39.451218,20.766767,http://db.edcs.eu/epigr/bilder.php?bilder.php?...,309 to 313,NaN,NaN,NaN,NaN,HD052964
9,EDCS-66200107,"AE 2008, 01264",Achaia,Aigio / Egio / Aiyion / Aegeum,[3]ν τὸν λαμ/[πρότατον ὑπατικ]όν Ἄππιος Κυ/[3 ...,ν τὸν λαμ όν Ἄππιος Κυ τρωνα,ν τὸν λαμπρότατον ὑπατικόν Ἄππιος Κυ τὸν ἑαυτο...,NaN,NaN,NaN,ν τὸν λαμ όν Ἄππιος Κυ τρωνα,ν τὸν λαμπρότατον ὑπατικόν Ἄππιος Κυ τὸν ἑαυτο...,NaN,lapis,http://db.edcs.eu/epigr/partner.php?s_language...,151.0,250.0,151.0,250.0,38.250714,22.085822,NaN,151 to 250,NaN,NaN,NaN,NaN,HD067316
10,EDCS-13800065,"CIL 01, 02955 = ILLRP 00370 (p 327) = ILGR 000...",Achaia,Aigio / Egio / Aiyion / Aegeum,Italicei / quei Aegei negotiantur / P(ublium) ...,Italicei quei Aegei negotiantur P Rutilium P f...,Italicei quei Aegei negotiantur Publium Rutili...,tituli honorarii,"[officium/professio, ordo senatorius, tria nom...",NaN,Italicei quei Aegei negotiantur P Rutilium P f...,Italicei quei Aegei negotiantur Publium Rutili...,officium/professio; ordo senatorius; tituli ...,lapis,http://db.edcs.eu/epigr/partner.php?s_language...,-74.0,-74.0,-74.0,-74.0,38.250714,22.085822,NaN,-74 to -74,NaN,NaN,NaN,NaN,HD017714
11,EDCS-66200108,"SEG-49, 00495a = AE 2008, 01265a",Achaia,Aigio / Egio / Aiyion / Aegeum,ἐπὶ Τιβ(ερίου) / Κλαυδί/ου Φρον/τείνου / Νεικη...,ἐπὶ Τιβ Κλαυδίου Φροντείνου Νεικηράτου ταμίου ...,ἐπὶ Τιβερίου Κλαυδίου Φροντείνου Νεικηράτου τα...,NaN,NaN,NaN,ἐπὶ Τιβ Κλαυδίου Φροντείνου Νεικηράτου ταμίου ...,ἐπὶ Τιβερίου Κλαυδίου Φροντείνου Νεικηράτου τα...,NaN,aes,http://db.edcs.eu/epigr/partner.php?s_language...,NaN,NaN,NaN,NaN,38.250714,22.085822,NaN,NaN,NaN,NaN,NaN,NaN,HD068315
12,EDCS-66200109,"SEG-49, 00495b = AE 2008, 01265b",Achaia,Aigio / Egio / Aiyion / Aegeum,ἐπὶ Τιβ(ερίου) / Κλαυδίου / Φροντείνου / Νεικη...,ἐπὶ Τιβ Κλαυδίου Φροντείνου Νεικηράτου ταμίου ...,ἐπὶ Τιβερίου Κλαυδίου Φροντείνου Νεικηράτου τα

In [50]:
len(EDCS[EDCS["EDH-ID"].notnull()])

80452

In [51]:
# actual length of EDCS
len(EDCS)

537262

In [102]:
EDH[EDH["EDH-ID"]=="HD057726"]

,commentary,country,depth,diplomatic_text,findspot_ancient,findspot_modern,height,EDH-ID,language,last_update,letter_size,literature,material,modern_region,not_after,not_before,responsible_individual,transcription,trismegistos_uri,type_of_inscription,type_of_monument,width,work_status,findspot,year_of_find,present_location,religion,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,people,province_label,pleiades_id,Longitude,Latitude,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,origdate_text_clean,clean_text_conservative,clean_text_interpretive_word,clean_text_interpretive_sentence
56898,(B): Δέκι<μ>ος statt Δέκιος.,Greece,NaN,ΔΕΚΙΟΣ ΛΕΙΒΙΟΣ ΞΕΥΞΙΣ / / D LEIVEI D LEIVEI SALVE,Sparta,Spárti,35 cm,HD057726,GL,2011-11-14,2 cm,"IG 05, 01, 00741. (B) ; ILLRP 0962. ; CIL 01 (...",NaN,Pelopónissos,NaN,NaN,Cowey,Δέκιος Λείβιος &lt;Ζ=Ξ&gt;εῦξις // D(eci) Leiv...,https://www.trismegistos.org/text/177460,epitaph,NaN,16 cm,no image,NaN,NaN,NaN,NaN,NaN,"[{'type': 'province', 'text': 'Achaia'}, {'typ...",\n \xce\x94\xe1\xbd\xb3\xce\xba\xce\xb9\xce\xb...,,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,92.0,epitaph,NaN,Achaia,570685,22.4309,37.0737,epitaph,Certain,35.0,16.0,NaN,NULL,NULL,NULL,Achaia,Certain,Greece,Certain,Sparta,Certain,Pelopónissos,Certain,Spárti,Certain,NULL,NULL,,Δέκιος Λείβιος lt Ξ gt εῦξις D Leivei D Leivei...,Δέκιος Λείβιος lt Ζ gt εῦξις Deci Leivei Deci ...,Δέκιος Λείβιος lt;Ζ gt;εῦξις Deci Leivei Deci ...


# Merge unfiltered versions of the two datasets

In [52]:
# rename and create some columns to make the dataset more similar
EDCS.rename(columns={"date_not_before" : "not_before", "date_not_after" : "not_after"}, inplace=True)
#EDCS["coordinates"] = EDCS.apply(lambda row: [row["Longitude"], row["Latitude"]], axis=1)

In [53]:
EDCS.rename(columns={"language" : "language_EDCS"}, inplace=True)
(EDCS["language_EDCS"].isnull()).sum()

528935

In [54]:
# save the raw edcs language
def get_language(edcs_language): # produce new language column in edh compatible form
    if str(edcs_language) == "nan":
        return "L"
    elif edcs_language == "GR":
        return "G"
    else:
        return np.nan
EDCS["language"] = EDCS["language_EDCS"].apply(get_language)

In [55]:
EDCS["language"].tolist()[:10]

['L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L']

In [56]:
(EDCS["language_EDCS"].isnull()).sum()

528935

In [57]:
(EDCS["language"]=="L").sum()

528935

In [58]:
EDCS["clean_text_interpretive_word_EDCS"] = EDCS["clean_text_interpretive_word"]

# Merge EDH & EDCS

In [59]:
LIST = pd.merge(EDCS, EDH, on="EDH-ID", how="outer")
len(LIST) # previously we had 505652

540607

In [60]:
len(LIST.columns) # previously we had 107

98

In [61]:
def choose_by_edh_id(row, col):
    if row["EDH-ID"] == None:
        value = row[col.rpartition("_y")[0] + "_x"] # use edcs value
    else:
        value = row[col]
    return value


for col in LIST.columns:
    if re.search("_y$", col):
        LIST[col.rpartition("_y")[0]] = LIST.apply(lambda row: choose_by_edh_id(row, col), axis=1)
        LIST.drop([col, col.rpartition("_y")[0] + "_x"], axis=1, inplace=True)
        #LIRE.rename(columns={col : col.rpartition("_y")[0]}, inplace=True)
len(LIST.columns)

90

In [62]:
LIST[LIST["EDH-ID"].notnull()].head(5)

,EDCS-ID,publication,province,place,inscription,inscription_conservative_cleaning,inscription_interpretive_cleaning,inscr_type,status_notation,inscr_process,status,partner_link,dating_from,dating_to,photo,raw_dating,language_EDCS,comment,extra_text,extra_html,EDH-ID,clean_text_interpretive_word_EDCS,commentary,country,depth,diplomatic_text,findspot_ancient,findspot_modern,height,last_update,letter_size,literature,modern_region,responsible_individual,transcription,trismegistos_uri,type_of_inscription,type_of_monument,width,work_status,findspot,year_of_find,present_location,religion,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,people,province_label,pleiades_id,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,origdate_text_clean,clean_text_interpretive_sentence,language,material,not_after,not_before,Longitude,Latitude,clean_text_conservative,clean_text_interpretive_word
456810,EDCS-24700151,"CIL 01, 02650 (p 1097) = IG-05-01, 00741 = ILL...",Achaia,Afesou,Δέκιος Λείβιος (Ζ)εῦξις // D(ecimi) Leivei D(e...,Δέκιος Λείβιος εῦξις D Leivei D Leivei salve,Δέκιος Λείβιος Ζεῦξις Decimi Leivei Decimi Lei...,tituli sepulcrales,"[praenomen et nomen, viri]",NaN,praenomen et nomen; tituli sepulcrales; viri,http://db.edcs.eu/epigr/partner.php?s_language...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HD057726,Δέκιος Λείβιος Ζεῦξις Decimi Leivei Decimi Lei...,(B): Δέκι<μ>ος statt Δέκιος.,Greece,NaN,ΔΕΚΙΟΣ ΛΕΙΒΙΟΣ ΞΕΥΞΙΣ / / D LEIVEI D LEIVEI SALVE,Sparta,Spárti,35 cm,2011-11-14,2 cm,"IG 05, 01, 00741. (B) ; ILLRP 0962. ; CIL 01 (...",Pelopónissos,Cowey,Δέκιος Λείβιος &lt;Ζ=Ξ&gt;εῦξις // D(eci) Leiv...,https://www.trismegistos.org/text/177460,epitaph,NaN,16 cm,no image,NaN,NaN,NaN,NaN,NaN,"[{'type': 'province', 'text': 'Achaia'}, {'typ...",\n \xce\x94\xe1\xbd\xb3\xce\xba\xce\xb9\xce\xb...,,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,92.0,epitaph,NaN,Achaia,570685,epitaph,Certain,35.0,16.0,NaN,NULL,NULL,NULL,Achaia,Certain,Greece,Certain,Sparta,Certain,Pelopónissos,Certain,Spárti,Certain,NULL,NULL,,Δέκιος Λείβιος lt;Ζ gt;εῦξις Deci Leivei Deci ...,GL,NaN,NaN,NaN,22.4309,37.0737,Δέκιος Λείβιος lt Ξ gt εῦξις D Leivei D Leivei...,Δέκιος Λείβιος lt Ζ gt εῦξις Deci Leivei Deci ...
456811,EDCS-24900077,"CIL 01, 00746 (p 944) = D 00867 = ILLRP 00374 ...",Achaia,Agia Triada / Merbaka / Midea,Q(uinto) Caecilio C(ai) f(ilio) Metel(l)o / im...,Q Caecilio C f Metelo imperatori Italici quei ...,Quinto Caecilio Cai filio Metello imperatori I...,tituli honorarii,"[officium/professio, ordo senatorius, tria nom...",NaN,officium/professio; ordo senatorius; tituli ...,http://db.edcs.eu/epigr/partner.php?s_language...,-68.0,-68.0,http://db.edcs.eu/epigr/bilder.php?bilder.php?...,-68 to -68,NaN,NaN,NaN,NaN,HD056163,Quinto Caecilio Cai filio Metello imperatori I...,NaN,Greece,NaN,Q CAECILIO C F METELO / IMPERATORI ITALICI / Q...,Midea,Midhéa,NaN,2011-11-11,NaN,"CIL 03, 00531. ; ILS 0867. ; CIL 01 (2. Aufl.)...",Pelopónissos,Cowey,Q(uinto) Caecilio C(ai) f(ilio) Metelo / imper...,https://www.trismegistos.org/text/177366,honorific inscription,NaN,NaN,no image,NaN,NaN,NaN,NaN,NaN,"[{'type': 'province', 'text': 'Achaia'}, {'typ...",\n Quinto Caecilio Cai filio Metelo imperatori...,68 BC,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,69.0,honorific inscription,NaN,Achaia,570485,honorific inscription,Certain,NaN,NaN,NaN,NULL,NULL,NULL,Achaia,Certain,Greece,Certain,Midea,Certain,Pelopónissos,Certain,Midhéa,Certain,NULL,NULL,68 BC,Quinto Caecilio Cai filio Metelo imperatori It...,L,NaN,-68.0,NaN

In [63]:
len(LIST)

540607

In [64]:
LIST.to_json("../data/large_data/LIST_unfiltered.json")

# FIlter by language

In [68]:
LIST = pd.read_json("../data/large_data/LIST_unfiltered.json")

In [66]:
(LIST["language"]=="L").sum()

529851

In [65]:
(LIST["language_EDCS"].isnull()).sum()

532280

In [73]:
# not classified as Latin in EDCS, but still in our dataset, since from EDH...
LIST[(LIST["language_EDCS"].isnull()) & (LIST["language"]!="L")]

,EDCS-ID,publication,province,place,inscription,inscription_conservative_cleaning,inscription_interpretive_cleaning,inscr_type,status_notation,inscr_process,status,partner_link,dating_from,dating_to,photo,raw_dating,language_EDCS,comment,extra_text,extra_html,EDH-ID,clean_text_interpretive_word_EDCS,commentary,country,depth,diplomatic_text,findspot_ancient,findspot_modern,height,last_update,letter_size,literature,modern_region,responsible_individual,transcription,trismegistos_uri,type_of_inscription,type_of_monument,width,work_status,findspot,year_of_find,present_location,religion,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,people,province_label,pleiades_id,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,origdate_text_clean,clean_text_interpretive_sentence,language,material,not_after,not_before,Longitude,Latitude,clean_text_conservative,clean_text_interpretive_word
456810,EDCS-24700151,"CIL 01, 02650 (p 1097) = IG-05-01, 00741 = ILL...",Achaia,Afesou,Δέκιος Λείβιος (Ζ)εῦξις // D(ecimi) Leivei D(e...,Δέκιος Λείβιος εῦξις D Leivei D Leivei salve,Δέκιος Λείβιος Ζεῦξις Decimi Leivei Decimi Lei...,tituli sepulcrales,"[praenomen et nomen, viri]",NaN,praenomen et nomen; tituli sepulcrales; viri,http://db.edcs.eu/epigr/partner.php?s_language...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HD057726,Δέκιος Λείβιος Ζεῦξις Decimi Leivei Decimi Lei...,(B): Δέκι<μ>ος statt Δέκιος.,Greece,NaN,ΔΕΚΙΟΣ ΛΕΙΒΙΟΣ ΞΕΥΞΙΣ / / D LEIVEI D LEIVEI SALVE,Sparta,Spárti,35 cm,2011-11-14,2 cm,"IG 05, 01, 00741. (B) ; ILLRP 0962. ; CIL 01 (...",Pelopónissos,Cowey,Δέκιος Λείβιος &lt;Ζ=Ξ&gt;εῦξις // D(eci) Leiv...,https://www.trismegistos.org/text/177460,epitaph,NaN,16 cm,no image,NaN,NaN,NaN,NaN,NaN,"[{'type': 'province', 'text': 'Achaia'}, {'typ...",\n \xce\x94\xe1\xbd\xb3\xce\xba\xce\xb9\xce\xb...,,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,92.0,epitaph,NaN,Achaia,570685,epitaph,Certain,35.0,16.0,NaN,NULL,NULL,NULL,Achaia,Certain,Greece,Certain,Sparta,Certain,Pelopónissos,Certain,Spárti,Certain,NULL,NULL,,Δέκιος Λείβιος lt;Ζ gt;εῦξις Deci Leivei Deci ...,GL,NaN,NaN,NaN,22.4309,37.0737,Δέκιος Λείβιος lt Ξ gt εῦξις D Leivei D Leivei...,Δέκιος Λείβιος lt Ζ gt εῦξις Deci Leivei Deci ...
456813,EDCS-66200107,"AE 2008, 01264",Achaia,Aigio / Egio / Aiyion / Aegeum,[3]ν τὸν λαμ/[πρότατον ὑπατικ]όν Ἄππιος Κυ/[3 ...,ν τὸν λαμ όν Ἄππιος Κυ τρωνα,ν τὸν λαμπρότατον ὑπατικόν Ἄππιος Κυ τὸν ἑαυτο...,NaN,NaN,NaN,NaN,http://db.edcs.eu/epigr/partner.php?s_language...,151.0,250.0,NaN,151 to 250,NaN,NaN,NaN,NaN,HD067316,ν τὸν λαμπρότατον ὑπατικόν Ἄππιος Κυ τὸν ἑαυτο...,NaN,Greece,2 cm,[ ]Ν ΤΟΝ ΛΑΜ / [ ]ΟΝ ΑΠΠΙΟΣ ΚΥ / [ ]ΤΡΩΝΑ,Aegeum,Aígion,15 cm,2014-01-31,3 cm,"AE 2008, 1264. ; A.D. Rizakis, Achaïe III. Les...",Dytikí Elláda,Cowey,[---]ν τὸν λαμ/[πρότατον ὑπατικ]όν Ἄππιος Κυ/[...,https://www.trismegistos.org/text/677832,NaN,NaN,34 cm,checked with photo,NaN,NaN,NaN,NaN,NaN,"[{'type': 'province', 'text': 'Achaia'}, {'typ...",\n \xce\xbd \xcf\x84\xe1\xbd\xb8\xce\xbd \xce\...,151 AD - 250 AD,NaN,NaN,NaN,NaN,48.0,"Marmor, wei\xc3\x9f",1000.0,143.0,unknown,NaN,Achaia,570049,NULL,NULL,15.0,34.0,2.0,Marble,NULL,NULL,Achaia,Certain,Greece,Certain,Aegeum,Certain,Dytikí Elláda,Certain,Aígion,Certain,NULL,NULL,151 AD - 250 AD,ν τὸν λαμπρότατον ὑπατικόν Ἄππιος Κυ τὸν ἑαυτο...,G,"Marmor, weiß",151.0,250.0,22.0845,38.2487,ν τὸν λαμόν Ἄππιος Κυτρωνα,ν τὸν λαμπρότατον ὑπατικόν Ἄππιος Κυ τὸν ἑαυτο...
456815,EDCS-66200108,"SEG-49, 00495a = AE 2008, 01265a",Achaia,Aigio / Egio / Aiyion / Aegeum,ἐ

In [103]:
# ok, filter for latin inscriptions only
LIST = LIST[LIST["language"]=="L"]

# Create geodataframes

In [104]:
LIST = gpd.GeoDataFrame(LIST, geometry=gpd.points_from_xy(LIST["Longitude"], LIST["Latitude"]), crs={"init": "epsg:3857"})

In [105]:
(LIST["Latitude"].isnull() | LIST["Longitude"].isnull()).sum()

28729

In [107]:
(LIST["Latitude"].notnull() | LIST["Longitude"].notnull()).sum()

501125

In [112]:
LIST["geotemporal?"] = (LIST["not_before"].notnull() | LIST["not_after"].notnull()) & (LIST["Latitude"].notnull() & LIST["Longitude"].notnull())

In [120]:
LIST["LIST-ID"] = ["LIST-" + str(el) for el in range(1,len(LIST)+1)]

In [149]:
# upload metadata
LIST_metadata = pd.read_csv("../data/LIST_2022.csv")
LIST_metadata.head(5)

,LI_unfiltered dataset,attribute_name,delete,merged,dataset_source,attribute_source,created_by_script,data_type,description
0,1,LIST-ID,NaN,NaN,SDAM,NaN,NaN,chracter,Unique ID of the LI dataset
1,2,geotemporal?,NaN,NaN,SDAM,NaN,NaN,bool,Whether the inscription has BOTH valid geocoor...
2,3,EDCS-ID,NaN,NaN,EDCS,http://db.edcs.eu/epigr/epikl_ergebnis.php,https://github.com/mqAncientHistory/EpigraphyS...,character,"Unique identifier, as provided by the EDCS; ht..."
3,4,publication,True,NaN,EDCS,http://db.edcs.eu/epigr/epikl_ergebnis.php,https://github.com/mqAncientHistory/EpigraphyS...,character,"Bibliographical references, as provided by the..."
4,5,province,NaN,NaN,EDCS,http://db.edcs.eu/epigr/epikl_ergebnis.php,https://github.com/mqAncientHistory/EpigraphyS...,character,Name of the province where the inscription was...


In [150]:
cols_to_remove = LIST_metadata[LIST_metadata["delete"]==True]["attribute_name"].tolist()
print(cols_to_remove)

['publication', 'dating_from', 'dating_to', 'inscription_conservative_cleaning', 'inscription_interpretive_cleaning', 'comment', 'photo', 'extra_text', 'extra_html', 'responsible_individual', 'literature', 'height', 'depth', 'material', 'type_of_monument', 'province_label', 'width', 'country', 'findspot_ancient', 'modern_region', 'findspot_modern', 'commentary', 'placenames_refs', 'origdate_text', 'layout_execution_text', 'support_objecttype_text', 'support_material_text', 'keywords_term_text', 'origdate_text_clean', 'findspot', 'religion']


In [152]:
LIST.drop(columns=cols_to_remove, inplace=True)
LIST.shape

# Tests with saving

In [234]:
LIST_sample = LIST.sample(10000)

In [235]:
#LIST_sample.set_index("LIST-ID", inplace=True)

In [236]:
LIST_sample

,EDCS-ID,province,place,inscription,inscr_type,status_notation,inscr_process,status,partner_link,raw_dating,language_EDCS,EDH-ID,clean_text_interpretive_word_EDCS,diplomatic_text,last_update,letter_size,transcription,trismegistos_uri,type_of_inscription,work_status,year_of_find,present_location,idno_tm,text_edition,layout_execution,support_objecttype,support_material,support_decoration,keywords_term,people,pleiades_id,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,clean_text_interpretive_sentence,language,not_after,not_before,Longitude,Latitude,clean_text_conservative,clean_text_interpretive_word,geometry,geotemporal?,LIST-ID
534469,EDCS-08400271,Sardinia,Mores / Hafa,M(ilia) p(assuum) CXVIIII / d(omino) n(ostro) ...,"miliaria, tituli operum","Augusti/Augustae, ordo equester, tria nomina, ...",NaN,Augusti/Augustae; miliaria; ordo equester; ...,http://db.edcs.eu/epigr/partner.php?s_language...,270 to 270,NaN,HD003010,Milia passuum CXVIIII domino nostro Imperatori...,M P CXVIIII / D N IMP CA / ES M AVRELIO / CLAV...,2012-03-29,3-5 cm,M(ilia) p(assuum) CXVIIII / d(omino) n(ostro) ...,https://www.trismegistos.org/text/175455,mile-/leaguestone,provisional,1980.0,"Sassari, Mus. Naz. G.A. Sanna",NaN,\n Milia passuum CXVIIII domino nostro Imperat...,NaN,89,83.0,1000.0,NaN,"[{'persname': {'name': [{'@type': 'praenomen',...",511337,mile-/leaguestone,Certain,142.0,50.0,NaN,Other,mile-/leaguestone,Certain,Sardinia,Certain,Italy,Certain,Olbia,Estimated,Sassari,Certain,Mores,Certain,"Località Silvaru, collina Monte Silvaru, bei",Certain,Milia passuum CXVIIII domino nostro Imperatori...,L,270.0,NaN,8.833300,40.550000,M p CXVIIII d n Imp Caes M Aurelio Claudio P F...,Milia passuum CXVIIII domino nostro Imperatori...,POINT (8.833 40.550),True,LIST-524035
87815,EDCS-50700832,Britannia,Beauport Park,Cla(ssis) Bri(tannica),tituli fabricationis,NaN,sigilla impressa,sigilla impressa; tituli fabricationis,NaN,NaN,NaN,None,Classis Britannica,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L,NaN,NaN,0.550443,50.893916,Cla Bri,Classis Britannica,POINT (0.550 50.894),False,LIST-86460
501964,EDCS-09100379,Hispania citerior,Sagunt / Sagunto / Saguntum,Sergiae M(arci) f(iliae) / Peregrinae / Theomn...,NaN,liberti/libertae,NaN,liberti/libertae,http://db.edcs.eu/epigr/partner.php?s_language...,71 to 130,NaN,HD025118,Sergiae Marci filiae Peregrinae Theomnestus et...,[ ] / [ ] / [ ] / POMPEIVS [ ] / BALONIVS SEVE...,2011-10-11,8.5-5.5 cm,[Antoniae L(uci) f(iliae)] / [Sergillae] / [--...,https://www.trismegistos.org/text/232656,epitaph,checked with photo,NaN,"Sagunto, Mus. Arq.",NaN,\n Antoniae Luci filiae Sergillae Pompeius Bal...,NaN,230,NaN,1000.0,92.0,"[{'persname': {'name': [{'@type': 'nomen', '#t...",266021,epitaph,Certain,56.0,125.0,39.0,NULL,grave monument,Certain,Hispania citerior,Certain,Spain,Certain,Saguntum,Certain,NULL,NULL,Sagunto,Certain,plaza de la Trinidad,Certain,Antoniae Luci filiae Sergillae Pompeius Baloni...,L,71.0,130.0,-0.278600,39.680000,Pompeius Balonius Sever Balonius Marcell Balon...,Antoniae Luci filiae Sergillae Pompeius Baloni...,POINT (-0.279 39.680),True,LIST-492532
523807,EDCS-10001125,Pannonia superior,Drnovo / Neviodunum,Su[r]us / Sironis f(ilius) / f(ecit) s(ibi) et...,tituli sepulcrales,"mulieres, nomen singulare, tria nomina, viri",NaN,mulieres; nomen singulare; tituli sepulcrale...,http://db.edcs.eu/epigr/partner.php?s_language...,71 to 130,NaN,HD068454,Surus Sironis filius fecit sibi et Claudiae Lo...,SV[ ]VS / SIRONIS F / F S ET C[ ] LONG / INIAE...,2014-11-20,7.5-

In [237]:
problematic_cols = []
for col in LIST_sample.columns:
    try:
        LIST_sample[[col, "geometry"]].to_file("../data/large_data/saving_test.geojson", driver="GeoJSON")
        print("ok: ", col)
    except:
        print("ko: ", col)
        if col != "geometry":
            problematic_cols.append(col)
!rm ../data/large_data/saving_test.geojson

ok:  EDCS-ID
ok:  province
ok:  place
ok:  inscription
ok:  inscr_type
ok:  status_notation
ko:  inscr_process
ok:  status
ok:  partner_link
ok:  raw_dating
ok:  language_EDCS
ok:  EDH-ID
ok:  clean_text_interpretive_word_EDCS
ok:  diplomatic_text
ok:  last_update
ok:  letter_size
ok:  transcription
ok:  trismegistos_uri
ok:  type_of_inscription
ok:  work_status
ok:  year_of_find
ok:  present_location
ok:  idno_tm
ok:  text_edition
ok:  layout_execution
ok:  support_objecttype
ok:  support_material
ok:  support_decoration
ok:  keywords_term
ok:  people
ok:  pleiades_id
ok:  type_of_inscription_clean
ok:  type_of_inscription_certainty
ok:  height_cm
ok:  width_cm
ok:  depth_cm
ok:  material_clean
ok:  type_of_monument_clean
ok:  type_of_monument_certainty
ok:  province_label_clean
ok:  province_label_certainty
ok:  country_clean
ok:  country_certainty
ok:  findspot_ancient_clean
ok:  findspot_ancient_certainty
ok:  modern_region_clean
ok:  modern_region_certainty
ok:  findspot_modern_cl

In [238]:
problematic_cols

['inscr_process']

In [232]:
LIST_sample["inscr_process"]

99008                  NaN
232740    sigilla impressa
79006                  NaN
157673                 NaN
376882                 NaN
                ...       
161018    sigilla impressa
274717                 NaN
214823                 NaN
439353                 NaN
52424     sigilla impressa
Name: inscr_process, Length: 10000, dtype: object

In [239]:
def list_to_str(val):
    if isinstance(val, list):
        val = ", ".join(val)
    return val
LIST_sample["inscr_type"] = LIST_sample["inscr_type"].apply(list_to_str)
LIST_sample["inscr_process"] = LIST_sample["inscr_process"].apply(list_to_str)
LIST_sample["status_notation"] = LIST_sample["status_notation"].apply(list_to_str)

In [240]:
def nan_to_empty_dict(val):
    if str(val) == "nan":
        val = []
    return str(val)
LIST_sample["people"] = LIST_sample["people"].apply(nan_to_empty_dict)

In [241]:
LIST_sample.to_file("../data/large_data/saving_test.geojson", driver="GeoJSON")
#!rm ../data/large_data/saving_test.geojson

In [218]:
LIST_sample_back = gpd.read_file("../data/large_data/saving_test.geojson", driver="GeoJSON")
LIST_sample_back.head(5)

,EDCS-ID,province,place,inscription,inscr_type,status_notation,inscr_process,status,partner_link,raw_dating,language_EDCS,EDH-ID,clean_text_interpretive_word_EDCS,diplomatic_text,last_update,letter_size,transcription,trismegistos_uri,type_of_inscription,work_status,year_of_find,present_location,idno_tm,text_edition,layout_execution,support_objecttype,support_material,support_decoration,keywords_term,people,pleiades_id,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,clean_text_interpretive_sentence,language,not_after,not_before,Longitude,Latitude,clean_text_conservative,clean_text_interpretive_word,geotemporal?,LIST-ID,geometry
0,EDCS-57300106,Provincia incerta,?,Dis Manibus bene / merenti Anicieto / coniugi ...,tituli sepulcrales,"mulieres, viri",None,mulieres; tituli sepulcrales; viri,None,None,None,None,Dis Manibus bene merenti Anicieto coniugi Secu...,None,None,None,None,None,None,None,NaN,None,None,None,None,None,NaN,NaN,NaN,[ ],None,None,None,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,L,NaN,NaN,NaN,NaN,Dis Manibus bene merenti Anicieto coniugi Secu...,Dis Manibus bene merenti Anicieto coniugi Secu...,False,LIST-288954,None
1,EDCS-19600443,Roma,Roma,Fort[unae] / Aug(ustae) / [,tituli sacri,None,None,tituli sacri,http://db.edcs.eu/epigr/partner.php?s_language...,1 to 200,None,None,Fortunae Augustae,None,None,None,None,None,None,None,NaN,None,None,None,None,None,NaN,NaN,NaN,[ ],None,None,None,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,L,200.0,1.0,12.482932,41.89332,Fort Aug,Fortunae Augustae,True,LIST-333975,POINT (12.48293 41.89332)
2,EDCS-11001731,Germania superior,Wiesbaden / Aquae Mattiacorum,[Hi]c quies/cit in pace / Runa qui v[ixit,"inscriptiones christianae, tituli sepulcrales",None,None,inscriptiones christianae; tituli sepulcrales,http://db.edcs.eu/epigr/partner.php?s_language...,401 to 550,None,HD060802,Hic quiescit in pace Runa qui vixit,[ ]C QVIES / [ ]IT IN PACE / RVNA QVI V,2019-05-09,2.0-3.0 cm,[Hi]c quies/[c]it in pace / Runa qui v(ixit),https://www.trismegistos.org/text/212402,epitaph,checked with photo,1873.0,"Wiesbaden, Stadtmus. am Markt",None,\n Hic quiescit in pace Runa qui vixit\n,None,260,60.0,2000.0,92.0,[ ],108750,epitaph,Certain,30.0,26.0,8.0,Limestone,slab,Certain,Germania superior,Certain,Germany,Certain,Aquae Mattiacorum (Civitas Mattiacorum),Certain,Hessen,Certain,Wiesbaden,Certain,"{Schwalbacher Straße} - {Luisenstraße}, zwischen",Certain,Hic quiescit in pace Runa qui vixit,L,401.0,550.0,8.238400,50.08340,c quiesit in pace Runa qui v,Hic quiescit in pace Runa qui vixit,True,LIST-488842,POINT (8.23840 50.08340)
3,EDCS-82300097,Provincia incerta,?,[Imp(erator) Caes(ar) divi Traiani Parthi]ci f...,diplomata militaria,"Augusti/Augustae, milites, ordo equester, ordo...",None,Augusti/Augustae; diplomata militaria; milit...,http://db.edcs.eu/epigr/partner.php?s_language...,129 to 129,None,HD055752,Imperator Caesar divi Traiani Parthici filius ...,[ ]CI F DIVI NER[ ]S AVG PONT [ ]OS III P P [ ...,2009-04-06,0.3 cm,[Imp(erator) Caes(ar) divi Traiani Parthi]ci f...,https://www.trismegistos.org/text/217740,military diploma,provisional,NaN,None,None,\n Imperator Caesar divi Traiani Parthici fili...,None,257,NaN,1000.0,91.0,[ ],,military diploma,Certain,4.0,4.0,0.0,NULL,tabula,Certain,unknown,Certain,unknown,Certain,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,Imperator Caesar divi Traiani Parthici filius ...,L,129.0,NaN,NaN,NaN,ci f divi Ners Aug pont os III p p in alis II ...,Imperator Caesar divi Traiani Parthici filius ...,False,LIST-517620,None
4,EDCS-15800185,D

In [222]:
LIST_sample.to_parquet("../data/large_data/saving_test.parquet", index=False)
#!rm ../data/large_data/saving_test.parquet

In [224]:
LIST_sample_back = gpd.read_parquet("../data/large_data/saving_test.parquet")
LIST_sample_back.head(5)

,EDCS-ID,province,place,inscription,inscr_type,status_notation,inscr_process,status,partner_link,raw_dating,language_EDCS,EDH-ID,clean_text_interpretive_word_EDCS,diplomatic_text,last_update,letter_size,transcription,trismegistos_uri,type_of_inscription,work_status,year_of_find,present_location,idno_tm,text_edition,layout_execution,support_objecttype,support_material,support_decoration,keywords_term,people,pleiades_id,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,clean_text_interpretive_sentence,language,not_after,not_before,Longitude,Latitude,clean_text_conservative,clean_text_interpretive_word,geometry,geotemporal?,LIST-ID
0,EDCS-57300106,Provincia incerta,?,Dis Manibus bene / merenti Anicieto / coniugi ...,tituli sepulcrales,"mulieres, viri",None,mulieres; tituli sepulcrales; viri,None,None,None,None,Dis Manibus bene merenti Anicieto coniugi Secu...,None,None,None,None,None,None,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,[],None,None,None,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,L,NaN,NaN,NaN,NaN,Dis Manibus bene merenti Anicieto coniugi Secu...,Dis Manibus bene merenti Anicieto coniugi Secu...,POINT EMPTY,False,LIST-288954
1,EDCS-19600443,Roma,Roma,Fort[unae] / Aug(ustae) / [,tituli sacri,None,None,tituli sacri,http://db.edcs.eu/epigr/partner.php?s_language...,1 to 200,None,None,Fortunae Augustae,None,None,None,None,None,None,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,[],None,None,None,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,L,200.0,1.0,12.482932,41.89332,Fort Aug,Fortunae Augustae,POINT (12.483 41.893),True,LIST-333975
2,EDCS-11001731,Germania superior,Wiesbaden / Aquae Mattiacorum,[Hi]c quies/cit in pace / Runa qui v[ixit,"inscriptiones christianae, tituli sepulcrales",None,None,inscriptiones christianae; tituli sepulcrales,http://db.edcs.eu/epigr/partner.php?s_language...,401 to 550,None,HD060802,Hic quiescit in pace Runa qui vixit,[ ]C QVIES / [ ]IT IN PACE / RVNA QVI V,2019-05-09,2.0-3.0 cm,[Hi]c quies/[c]it in pace / Runa qui v(ixit),https://www.trismegistos.org/text/212402,epitaph,checked with photo,1873.0,"Wiesbaden, Stadtmus. am Markt",NaN,\n Hic quiescit in pace Runa qui vixit\n,NaN,260,60.0,2000.0,92.0,[],108750,epitaph,Certain,30.0,26.0,8.0,Limestone,slab,Certain,Germania superior,Certain,Germany,Certain,Aquae Mattiacorum (Civitas Mattiacorum),Certain,Hessen,Certain,Wiesbaden,Certain,"{Schwalbacher Straße} - {Luisenstraße}, zwischen",Certain,Hic quiescit in pace Runa qui vixit,L,401.0,550.0,8.238400,50.08340,c quiesit in pace Runa qui v,Hic quiescit in pace Runa qui vixit,POINT (8.238 50.083),True,LIST-488842
3,EDCS-82300097,Provincia incerta,?,[Imp(erator) Caes(ar) divi Traiani Parthi]ci f...,diplomata militaria,"Augusti/Augustae, milites, ordo equester, ordo...",None,Augusti/Augustae; diplomata militaria; milit...,http://db.edcs.eu/epigr/partner.php?s_language...,129 to 129,None,HD055752,Imperator Caesar divi Traiani Parthici filius ...,[ ]CI F DIVI NER[ ]S AVG PONT [ ]OS III P P [ ...,2009-04-06,0.3 cm,[Imp(erator) Caes(ar) divi Traiani Parthi]ci f...,https://www.trismegistos.org/text/217740,military diploma,provisional,NaN,None,NaN,\n Imperator Caesar divi Traiani Parthici fili...,NaN,257,NaN,1000.0,91.0,[],,military diploma,Certain,4.0,4.0,0.0,NULL,tabula,Certain,unknown,Certain,unknown,Certain,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,Imperator Caesar divi Traiani Parthici filius ...,L,129.0,NaN,NaN,NaN,ci f divi Ners Aug pont os III p p in alis II ...,Imperator Caesar divi Traiani Parthici filius ...,POINT EMPTY,False,LIST-517620
4,EDCS-15800185,Dacia,A

In [242]:
# apply to the whole dataset
LIST["inscr_type"] = LIST["inscr_type"].apply(list_to_str)
LIST["inscr_process"] = LIST["inscr_process"].apply(list_to_str)
LIST["status_notation"] = LIST["status_notation"].apply(list_to_str)
LIST["people"] = LIST["people"].apply(nan_to_empty_dict)

In [243]:
#LIST.to_file("../data/large_data/LIST_v0.2.geojson", driver="GeoJSON")

# Finetuning of the unfiltered LIST dataset before saving

In [23]:
LIST = gpd.read_file("../data/large_data/LIST_working.geojson", driver="GeoJSON")

In [24]:
cert_cols = [col for  col in LIST.columns if "certainty" in col]
cert_cols

['type_of_inscription_certainty',
 'type_of_monument_certainty',
 'province_label_certainty',
 'country_certainty',
 'findspot_ancient_certainty',
 'modern_region_certainty',
 'findspot_modern_certainty',
 'findspot_certainty']

In [25]:
LIST[cert_cols+ ["geometry"]].to_feather("../data/LIST_test_cert.feather")

In [26]:
for col in cert_cols:
    LIST[col] = LIST[col].apply(lambda x: "certain" in str(x))

In [29]:
LIST.to_feather("../data/large_data/LIST_merged.feather", index=False)

In [3]:
LIST = gpd.read_feather("../data/large_data/LIST_merged.feather")
LIST.head(5)

,EDCS-ID,province,place,inscription,inscr_type,status_notation,inscr_process,status,partner_link,raw_dating,language_EDCS,EDH-ID,clean_text_interpretive_word_EDCS,diplomatic_text,last_update,letter_size,transcription,trismegistos_uri,type_of_inscription,work_status,year_of_find,present_location,idno_tm,text_edition,layout_execution,support_objecttype,support_material,support_decoration,keywords_term,people,pleiades_id,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,clean_text_interpretive_sentence,language,not_after,not_before,Longitude,Latitude,clean_text_conservative,clean_text_interpretive_word,geotemporal?,LIST-ID,geometry
0,EDCS-31400030,Achaia,?,Leius,None,None,None,None,None,None,None,None,Leius,None,None,None,None,None,None,None,NaN,None,None,None,None,None,NaN,NaN,NaN,[ ],None,None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,None,L,NaN,NaN,NaN,NaN,Leius,Leius,False,LIST-1,None
1,EDCS-55701593,Achaia,Agios Donatos / Photike,Cn(aeus) Atei(us),tituli fabricationis,"praenomen et nomen, viri",sigilla impressa,praenomen et nomen; sigilla impressa; tituli...,None,None,None,None,Cnaeus Ateius,None,None,None,None,None,None,None,NaN,None,None,None,None,None,NaN,NaN,NaN,[ ],None,None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,None,L,NaN,NaN,20.506908,39.475976,Cn Atei,Cnaeus Ateius,False,LIST-2,POINT (20.50691 39.47598)
2,EDCS-55701594,Achaia,Agios Donatos / Photike,Avill(ius),tituli fabricationis,"nomen singulare, viri",sigilla impressa,nomen singulare; sigilla impressa; tituli fa...,None,None,None,None,Avillius,None,None,None,None,None,None,None,NaN,None,None,None,None,None,NaN,NaN,NaN,[ ],None,None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,None,L,NaN,NaN,20.506908,39.475976,Avill,Avillius,False,LIST-3,POINT (20.50691 39.47598)
3,EDCS-55701595,Achaia,Agios Donatos / Photike,L(ucius) Av(illius),tituli fabricationis,"praenomen et nomen, viri",sigilla impressa,praenomen et nomen; sigilla impressa; tituli...,None,None,None,None,Lucius Avillius,None,None,None,None,None,None,None,NaN,None,None,None,None,None,NaN,NaN,NaN,[ ],None,None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,None,L,NaN,NaN,20.506908,39.475976,L Av,Lucius Avillius,False,LIST-4,POINT (20.50691 39.47598)
4,EDCS-55701596,Achaia,Agios Donatos / Photike,Camurius,tituli fabricationis,"nomen singulare, viri",sigilla impressa,nomen singulare; sigilla impressa; tituli fa...,None,None,None,None,Camurius,None,None,None,None,None,None,None,NaN,None,None,None,None,None,NaN,NaN,NaN,[ ],None,None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,None,L,NaN,NaN,20.506908,39.475976,Camurius,Camurius,False,LIST-5,POINT (20.50691 39.47598)


In [4]:
len(LIST)

529851

In [5]:
(LIST["geotemporal?"]).sum()

188896

In [6]:
s = sddk.cloudSession(provider="sciencedata.dk", shared_folder_name="SDAM_root", owner="648597@au.dk")

connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [7]:
s.write_file("SDAM_data/LI/public/LIST_merged_2022-11-11.feather", LIST)

Your <class 'geopandas.geodataframe.GeoDataFrame'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/LI/public/LIST_merged_2022-11-11.feather"


In [8]:
s.write_file("SDAM_data/LI/public/LIST_merged_2022-11-11.geojson", LIST)

Your <class 'geopandas.geodataframe.GeoDataFrame'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/LI/public/LIST_merged_2022-11-11.geojson"


# Reordering columns and filtering

In [3]:
LIST = gpd.read_feather("../data/large_data/LIST_merged.feather")
LIST.head(5)

,EDCS-ID,province,place,inscription,inscr_type,status_notation,inscr_process,status,partner_link,raw_dating,language_EDCS,EDH-ID,clean_text_interpretive_word_EDCS,diplomatic_text,last_update,letter_size,transcription,trismegistos_uri,type_of_inscription,work_status,year_of_find,present_location,idno_tm,text_edition,layout_execution,support_objecttype,support_material,support_decoration,keywords_term,people,pleiades_id,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,clean_text_interpretive_sentence,language,not_after,not_before,Longitude,Latitude,clean_text_conservative,clean_text_interpretive_word,geotemporal?,LIST-ID,geometry
0,EDCS-31400030,Achaia,?,Leius,None,None,None,None,None,None,None,None,Leius,None,None,None,None,None,None,None,NaN,None,None,None,None,None,NaN,NaN,NaN,[ ],None,None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,None,L,NaN,NaN,NaN,NaN,Leius,Leius,False,LIST-1,None
1,EDCS-55701593,Achaia,Agios Donatos / Photike,Cn(aeus) Atei(us),tituli fabricationis,"praenomen et nomen, viri",sigilla impressa,praenomen et nomen; sigilla impressa; tituli...,None,None,None,None,Cnaeus Ateius,None,None,None,None,None,None,None,NaN,None,None,None,None,None,NaN,NaN,NaN,[ ],None,None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,None,L,NaN,NaN,20.506908,39.475976,Cn Atei,Cnaeus Ateius,False,LIST-2,POINT (20.50691 39.47598)
2,EDCS-55701594,Achaia,Agios Donatos / Photike,Avill(ius),tituli fabricationis,"nomen singulare, viri",sigilla impressa,nomen singulare; sigilla impressa; tituli fa...,None,None,None,None,Avillius,None,None,None,None,None,None,None,NaN,None,None,None,None,None,NaN,NaN,NaN,[ ],None,None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,None,L,NaN,NaN,20.506908,39.475976,Avill,Avillius,False,LIST-3,POINT (20.50691 39.47598)
3,EDCS-55701595,Achaia,Agios Donatos / Photike,L(ucius) Av(illius),tituli fabricationis,"praenomen et nomen, viri",sigilla impressa,praenomen et nomen; sigilla impressa; tituli...,None,None,None,None,Lucius Avillius,None,None,None,None,None,None,None,NaN,None,None,None,None,None,NaN,NaN,NaN,[ ],None,None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,None,L,NaN,NaN,20.506908,39.475976,L Av,Lucius Avillius,False,LIST-4,POINT (20.50691 39.47598)
4,EDCS-55701596,Achaia,Agios Donatos / Photike,Camurius,tituli fabricationis,"nomen singulare, viri",sigilla impressa,nomen singulare; sigilla impressa; tituli fa...,None,None,None,None,Camurius,None,None,None,None,None,None,None,NaN,None,None,None,None,None,NaN,NaN,NaN,[ ],None,None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,None,L,NaN,NaN,20.506908,39.475976,Camurius,Camurius,False,LIST-5,POINT (20.50691 39.47598)


In [4]:
# reorder and filter out some other useless columns

LIST = LIST[['LIST-ID', 'EDCS-ID', 'EDH-ID','trismegistos_uri',  'pleiades_id', 'transcription', 'inscription', 'clean_text_conservative', 'clean_text_interpretive_sentence','clean_text_interpretive_word', 'clean_text_interpretive_word_EDCS', 'diplomatic_text', 'province', 'place', 'inscr_type',
 'status_notation', 'inscr_process', 'status', 'partner_link', 'last_update',
 'letter_size', 'type_of_inscription', 'work_status', 'year_of_find',
 'present_location', 'text_edition', 'support_objecttype', 'support_material', 'support_decoration',
 'keywords_term', 'people', 'type_of_inscription_clean',
 'type_of_inscription_certainty', 'height_cm', 'width_cm', 'depth_cm',
 'material_clean', 'type_of_monument_clean',
 'type_of_monument_certainty', 'province_label_clean',
 'province_label_certainty', 'country_clean', 'country_certainty',
 'findspot_ancient_clean', 'findspot_ancient_certainty',
 'modern_region_clean', 'modern_region_certainty',
 'findspot_modern_clean', 'findspot_modern_certainty', 'findspot_clean',
 'findspot_certainty', 'language', 'language_EDCS', 'raw_dating',
 'not_after', 'not_before', 'Longitude', 'Latitude',
 'geotemporal?', 'geometry']]

In [5]:
sum(LIST["EDH-ID"].notnull())

81266

In [6]:
sum(LIST["geometry"].notnull())

501122

In [7]:
sum(LIST["not_after"].notnull())

200433

In [8]:
sum(LIST["not_before"].notnull())


197066

In [9]:
sum((LIST["not_after"].notnull() | (LIST["not_before"].notnull())))


200433

In [10]:
sum((LIST["geometry"].notnull()) & ((LIST["not_after"].notnull() | (LIST["not_before"].notnull()))))

188896

In [11]:
LIST["geotemporal?"].sum()

188896

In [51]:
#LIST = LIST[(LIST["geometry"].notnull()) & ((LIST["not_after"].notnull() | (LIST["not_before"].notnull$())))]

In [70]:
#LIST.drop(columns=["geotemporal?"], inplace=True)

# Map inscriptions on the Empire and its provinces

In [12]:
pleiades_regions = gpd.read_file("../data/pleiades_regions.geojson", driver="GeoJSON")
pleiades_regions = pleiades_regions.to_crs(3035)
pleiades_regions_merged = pleiades_regions.unary_union
pleiades_regions_merged_buffer = pleiades_regions_merged.buffer(distance=5000)

In [14]:
medsea = gpd.read_file("https://geo.vliz.be/geoserver/wfs?request=getfeature&service=wfs&version=1.1.0&typename=MarineRegions:iho&outputformat=json&filter=%3COr%3E%3COr%3E%3COr%3E%3COr%3E%3COr%3E%3COr%3E%3COr%3E%3COr%3E%3COr%3E%3CPropertyIsEqualTo%3E%3CPropertyName%3Emrgid%3C%2FPropertyName%3E%3CLiteral%3E3314%3C%2FLiteral%3E%3C%2FPropertyIsEqualTo%3E%3CPropertyIsEqualTo%3E%3CPropertyName%3Emrgid%3C%2FPropertyName%3E%3CLiteral%3E3315%3C%2FLiteral%3E%3C%2FPropertyIsEqualTo%3E%3C%2FOr%3E%3CPropertyIsEqualTo%3E%3CPropertyName%3Emrgid%3C%2FPropertyName%3E%3CLiteral%3E3322%3C%2FLiteral%3E%3C%2FPropertyIsEqualTo%3E%3C%2FOr%3E%3CPropertyIsEqualTo%3E%3CPropertyName%3Emrgid%3C%2FPropertyName%3E%3CLiteral%3E3324%3C%2FLiteral%3E%3C%2FPropertyIsEqualTo%3E%3C%2FOr%3E%3CPropertyIsEqualTo%3E%3CPropertyName%3Emrgid%3C%2FPropertyName%3E%3CLiteral%3E3346%3C%2FLiteral%3E%3C%2FPropertyIsEqualTo%3E%3C%2FOr%3E%3CPropertyIsEqualTo%3E%3CPropertyName%3Emrgid%3C%2FPropertyName%3E%3CLiteral%3E3351%3C%2FLiteral%3E%3C%2FPropertyIsEqualTo%3E%3C%2FOr%3E%3CPropertyIsEqualTo%3E%3CPropertyName%3Emrgid%3C%2FPropertyName%3E%3CLiteral%3E3363%3C%2FLiteral%3E%3C%2FPropertyIsEqualTo%3E%3C%2FOr%3E%3CPropertyIsEqualTo%3E%3CPropertyName%3Emrgid%3C%2FPropertyName%3E%3CLiteral%3E3386%3C%2FLiteral%3E%3C%2FPropertyIsEqualTo%3E%3C%2FOr%3E%3CPropertyIsEqualTo%3E%3CPropertyName%3Emrgid%3C%2FPropertyName%3E%3CLiteral%3E4279%3C%2FLiteral%3E%3C%2FPropertyIsEqualTo%3E%3C%2FOr%3E%3CPropertyIsEqualTo%3E%3CPropertyName%3Emrgid%3C%2FPropertyName%3E%3CLiteral%3E4280%3C%2FLiteral%3E%3C%2FPropertyIsEqualTo%3E%3C%2FOr%3E", driver="GeoJSON")
medsea = medsea.to_crs(3035)
medsea_merged = medsea.unary_union
medsea_merged_buffer = medsea_merged.buffer(distance=5000)

In [15]:
pleiades_regions = gpd.read_file('../data/pleiades_regions.geojson', driver='GeoJSON')
# merge the regions into one multipolygon
RE_merged = pleiades_regions.unary_union

In [16]:
pleiades_regions["name"].tolist()[:10]

['Lusitania',
 'Byzacena',
 'Viennensis',
 'Lycia et Pamphylia',
 'Savia',
 'Numidia Cirtensis',
 'Numidia',
 'Noricum',
 'Galatia',
 'Moesia Inferior']

In [17]:
%%time
LIST["withinRE?"] = LIST.within(RE_merged)

CPU times: user 21min 32s, sys: 12.9 s, total: 21min 44s
Wall time: 22min 4s


In [29]:
len(LIST)

529851

In [30]:
LIST["withinRE?"].sum()

481498

# Urban contexts

In [20]:
roman_cities = gpd.read_file("../data/roman_cities_pop.geojson")

In [21]:
roman_cities.sort_values("pop_est", ascending=False, inplace=True)
roman_cities.head(5)

,Primary Key,Ancient Toponym,Modern Toponym,Province,Country,Barrington Atlas Rank,Barrington Atlas Reference,Start Date,End Date,Select Bibliography,Area,Estimate Basis,num_of_structures,ID,level_0,Unnamed: 0_y,index,pop_sample,pop_est,pop_diff,geometry
991,Hanson2016_992,Roma,Rome,Italia (VII Etruria),Italy,1,43 B2,-700,NaN,Beloch 1886; Benario 1981; de Ligt 2012; Golvi...,1783.0,Fourteen Augustan regions,236,992,89.0,89.0,89.0,923406.0,923313.0,1.000101,POINT (12.48510 41.89278)
109,Hanson2016_110,Alexandria (Aegyptus),Alexandria,Aegyptus,Egypt,1,74 B2,-322,NaN,Beloch 1886; Bowman 2011; Cohen 2006; Humphrey...,972.0,Urban grid,30,110,3.0,3.0,3.0,410535.0,410502.0,1.000080,POINT (29.90373 31.19502)
179,Hanson2016_180,Lepcis Magna,Lebda,Africa Proconsularis,Libya,1,35 G2,-30,NaN,Bomgardner 2000; Golvin 1988; Humphrey 1986; L...,466.0,Ramparts,46,180,NaN,NaN,NaN,NaN,153722.0,NaN,POINT (14.29306 32.63833)
308,Hanson2016_309,Rhodus,Rhodes,Asia,Greece,3,60 G3,-408,NaN,Beloch 1886; Hansen 2006; Hansen and Nielsen 2...,412.0,Urban grid,17,309,NaN,NaN,NaN,NaN,130398.0,NaN,POINT (28.22701 36.44324)
1323,Hanson2016_1324,Antiochia (Syria),Antakya,Syria,Turkey,1,67 C4,-300,NaN,Boatwright 2000; Jones 1937; McEvedy 2011; Nel...,399.0,"Walls of Theodosius II, Justinian, and topography",33,1324,NaN,NaN,NaN,NaN,124930.0,NaN,POINT (36.16963 36.21017)


In [22]:
sum(roman_cities["pop_est"] < 1000)

0

In [23]:
cities_id_name_dict = dict(zip(roman_cities["ID"], roman_cities["Ancient Toponym"]))
cities_id_pop_dict = dict(zip(roman_cities["ID"], roman_cities["pop_est"]))


In [24]:
len(roman_cities)

1388

In [25]:
# total urban population
roman_cities["pop_est"].sum()

10214337.0

In [26]:
roman_cities.columns

Index(['Primary Key', 'Ancient Toponym', 'Modern Toponym', 'Province',
       'Country', 'Barrington Atlas Rank', 'Barrington Atlas Reference',
       'Start Date', 'End Date', 'Select Bibliography', 'Area',
       'Estimate Basis', 'num_of_structures', 'ID', 'level_0', 'Unnamed: 0_y',
       'index', 'pop_sample', 'pop_est', 'pop_diff', 'geometry'],
      dtype='object')

In [27]:
masks_dict = {}
masks_dict["large"] = roman_cities["pop_est"] >= 30000
masks_dict["medium"] = (roman_cities["pop_est"] >= 5000) & (roman_cities["pop_est"] < 30000)
masks_dict["small"] = small_cities_mask = roman_cities["pop_est"] < 5000

In [28]:
for mask in masks_dict.keys():
    print(mask, len(roman_cities[masks_dict[mask]]), roman_cities[masks_dict[mask]]["pop_est"].sum())

large 68 4641352.0
medium 337 4005250.0
small 983 1567735.0


In [81]:
geod = Geod(ellps="WGS84")
def get_dist_m(inscr_geo, city_geo):
    try:
        line_string = LineString([inscr_geo, city_geo])
        return np.round(geod.geometry_length(line_string), 2)
    except:
        return None

In [61]:
len(LIST)

529851

In [91]:
def ckdnearest(city_type):
    cities_subset = roman_cities[masks_dict[city_type]]
    incr_ids = LIST[LIST.geometry.notnull()]["LIST-ID"]
    insr_geometries = np.array(list(LIST[LIST.geometry.notnull()].geometry.apply(lambda x: (x.x, x.y))))
    cities_geometries = np.array(list(cities_subset.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(cities_geometries)
    dist, idx = btree.query(insr_geometries, k=1)
    inscr_cities_name_dict = dict(zip(incr_ids, cities_subset.iloc[idx]["Ancient Toponym"]))
    inscr_cities_pop_dict = dict(zip(incr_ids, cities_subset.iloc[idx]["pop_est"]))
    inscr_cities_geo_dict = dict(zip(incr_ids, cities_subset.iloc[idx].geometry))
    incr_ids_missing = LIST[LIST.geometry.isnull()]["LIST-ID"]
    for id in incr_ids_missing:
        inscr_cities_name_dict[id] = None
        inscr_cities_pop_dict[id] = None
        inscr_cities_geo_dict[id] = None
    LIST[city_type + "_city_name"] = LIST["LIST-ID"].apply(lambda x: inscr_cities_name_dict[x])
    LIST[city_type + "_city_geo"] = LIST["LIST-ID"].apply(lambda x: inscr_cities_geo_dict[x])
    LIST[city_type + "_city_pop"] = LIST["LIST-ID"].apply(lambda x: inscr_cities_pop_dict[x])
    LIST[city_type + "_city_dist"] = LIST.apply(lambda row: get_dist_m(row.geometry, row[city_type + "_city_geo"]), axis=1)

In [92]:
ckdnearest("large")
ckdnearest("medium")
ckdnearest("small")

In [93]:
LIST.head(10)

,LIST-ID,EDCS-ID,EDH-ID,trismegistos_uri,pleiades_id,transcription,inscription,clean_text_conservative,clean_text_interpretive_sentence,clean_text_interpretive_word,clean_text_interpretive_word_EDCS,diplomatic_text,province,place,inscr_type,status_notation,inscr_process,status,partner_link,last_update,letter_size,type_of_inscription,work_status,year_of_find,present_location,text_edition,support_objecttype,support_material,support_decoration,keywords_term,people,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,language,language_EDCS,raw_dating,not_after,not_before,Longitude,Latitude,geotemporal?,geometry,withinRE?,large_city_name,large_city_geo,large_city_dist,large_city_pop,medium_city_name,medium_city_geo,medium_city_pop,medium_city_dist,small_city_name,small_city_geo,small_city_pop,small_city_dist
0,LIST-1,EDCS-31400030,None,None,None,None,Leius,Leius,None,Leius,Leius,None,Achaia,?,None,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,NaN,NaN,False,None,False,None,None,NaN,NaN,None,None,NaN,NaN,None,None,NaN,NaN
1,LIST-2,EDCS-55701593,None,None,None,None,Cn(aeus) Atei(us),Cn Atei,None,Cnaeus Ateius,Cnaeus Ateius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"praenomen et nomen, viri",sigilla impressa,praenomen et nomen; sigilla impressa; tituli...,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.50691 39.47598),True,Nicopolis (Achaea),POINT (20.733395 39.026505),53593.41,41221.0,Ambracia,POINT (20.987 39.158),27923.0,54409.47,Dodona,POINT (20.787767 39.546432),1000.0,25388.88
2,LIST-3,EDCS-55701594,None,None,None,None,Avill(ius),Avill,None,Avillius,Avillius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"nomen singulare, viri",sigilla impressa,nomen singulare; sigilla impressa; tituli fa...,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.50691 39.47598),True,Nicopolis (Achaea),POINT (20.733395 39.026505),53593.41,41221.0,Ambracia,POINT (20.987 39.158),27923.0,54409.47,Dodona,POINT (20.787767 39.546432),1000.0,25388.88
3,LIST-4,EDCS-55701595,None,None,None,None,L(ucius) Av(illius),L Av,None,Lucius Avillius,Lucius Avillius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"praenomen et nomen, viri",sigilla impressa,praenomen et nomen; sigilla impressa; tituli...,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.50691 39.47598),True,Nicopolis (Achaea),POINT (20.733395 39.026505),53593.41,41221.0,Ambracia,POINT (20.987 39.158),27923.0,54409.47,Dodona,POINT (20.787767 39.546432),1000.0,25388.88
4,LIST-5,EDCS-55701596,None,None,None,None,Camurius,Camurius,None,Camurius,Camurius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"nomen singulare, viri",sigilla impressa,nomen singulare; sigilla impressa; tituli fa...,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.50691 39.47598),True,Nicopolis (Achaea),POINT (20.733395 39.02650

In [102]:
def get_urban_context(row):
    urban_context_city = None
    urban_context_pop_est = None
    if row.geometry == None:
        urban_context = None
    else:
        urban_context = "rural"
        buffer = 5000
        for context in ["large", "medium", "small"]:
            if row[context + "_city_dist"] <= buffer:
                urban_context = context
                urban_context_city = row[context + "_city_name"]
                urban_context_pop_est = row[context + "_city_pop"]
                break
    return urban_context, urban_context_city, urban_context_pop_est

urban_contexts = LIST.apply(lambda row: get_urban_context(row), axis=1)

LIST["urban_context"] = [el[0] for el in urban_contexts]
LIST["urban_context_city"] = [el[1] for el in urban_contexts]
LIST["urban_context_pop_est"] = [el[2] for el in urban_contexts]

In [108]:
contexts = ["large", "medium", "small", "rural"]
for context in contexts:
    print(context, (LIST["urban_context"]==context).sum())

large 156724
medium 131307
small 81345
rural 131746


In [107]:
(LIST["urban_context"].isnull()).sum()

28729

In [105]:
LIST

,LIST-ID,EDCS-ID,EDH-ID,trismegistos_uri,pleiades_id,transcription,inscription,clean_text_conservative,clean_text_interpretive_sentence,clean_text_interpretive_word,clean_text_interpretive_word_EDCS,diplomatic_text,province,place,inscr_type,status_notation,inscr_process,status,partner_link,last_update,letter_size,type_of_inscription,work_status,year_of_find,present_location,text_edition,support_objecttype,support_material,support_decoration,keywords_term,people,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,language,language_EDCS,raw_dating,not_after,not_before,Longitude,Latitude,geotemporal?,geometry,withinRE?,large_city_name,large_city_geo,large_city_dist,large_city_pop,medium_city_name,medium_city_geo,medium_city_pop,medium_city_dist,small_city_name,small_city_geo,small_city_pop,small_city_dist,urban_context,urban_context_city,urban_context_pop_est
0,LIST-1,EDCS-31400030,None,None,None,None,Leius,Leius,None,Leius,Leius,None,Achaia,?,None,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,NaN,NaN,False,None,False,None,None,NaN,NaN,None,None,NaN,NaN,None,None,NaN,NaN,None,None,NaN
1,LIST-2,EDCS-55701593,None,None,None,None,Cn(aeus) Atei(us),Cn Atei,None,Cnaeus Ateius,Cnaeus Ateius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"praenomen et nomen, viri",sigilla impressa,praenomen et nomen; sigilla impressa; tituli...,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.50691 39.47598),True,Nicopolis (Achaea),POINT (20.733395 39.026505),53593.41,41221.0,Ambracia,POINT (20.987 39.158),27923.0,54409.47,Dodona,POINT (20.787767 39.546432),1000.0,25388.88,rural,None,NaN
2,LIST-3,EDCS-55701594,None,None,None,None,Avill(ius),Avill,None,Avillius,Avillius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"nomen singulare, viri",sigilla impressa,nomen singulare; sigilla impressa; tituli fa...,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.50691 39.47598),True,Nicopolis (Achaea),POINT (20.733395 39.026505),53593.41,41221.0,Ambracia,POINT (20.987 39.158),27923.0,54409.47,Dodona,POINT (20.787767 39.546432),1000.0,25388.88,rural,None,NaN
3,LIST-4,EDCS-55701595,None,None,None,None,L(ucius) Av(illius),L Av,None,Lucius Avillius,Lucius Avillius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"praenomen et nomen, viri",sigilla impressa,praenomen et nomen; sigilla impressa; tituli...,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.50691 39.47598),True,Nicopolis (Achaea),POINT (20.733395 39.026505),53593.41,41221.0,Ambracia,POINT (20.987 39.158),27923.0,54409.47,Dodona,POINT (20.787767 39.546432),1000.0,25388.88,rural,None,NaN
4,LIST-5,EDCS-55701596,None,None,None,None,Camurius,Camurius,None,Camurius,Camurius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"nomen singulare, viri",sigilla impressa,nomen singulare; sigilla impressa; tituli fa...,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,

In [110]:
# perhaps thesecolumns could be useful for some analysis, but not now...

LIST.drop(columns=['large_city_name', 'large_city_geo', 'large_city_dist', 'medium_city_name', 'medium_city_geo', 'medium_city_dist', 'small_city_name', 'small_city_geo','small_city_dist'], inplace=True)

In [111]:
LIST

,LIST-ID,EDCS-ID,EDH-ID,trismegistos_uri,pleiades_id,transcription,inscription,clean_text_conservative,clean_text_interpretive_sentence,clean_text_interpretive_word,clean_text_interpretive_word_EDCS,diplomatic_text,province,place,inscr_type,status_notation,inscr_process,status,partner_link,last_update,letter_size,type_of_inscription,work_status,year_of_find,present_location,text_edition,support_objecttype,support_material,support_decoration,keywords_term,people,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,language,language_EDCS,raw_dating,not_after,not_before,Longitude,Latitude,geotemporal?,geometry,withinRE?,large_city_pop,medium_city_pop,small_city_pop,urban_context,urban_context_city,urban_context_pop_est
0,LIST-1,EDCS-31400030,None,None,None,None,Leius,Leius,None,Leius,Leius,None,Achaia,?,None,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,NaN,NaN,False,None,False,NaN,NaN,NaN,None,None,NaN
1,LIST-2,EDCS-55701593,None,None,None,None,Cn(aeus) Atei(us),Cn Atei,None,Cnaeus Ateius,Cnaeus Ateius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"praenomen et nomen, viri",sigilla impressa,praenomen et nomen; sigilla impressa; tituli...,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.50691 39.47598),True,41221.0,27923.0,1000.0,rural,None,NaN
2,LIST-3,EDCS-55701594,None,None,None,None,Avill(ius),Avill,None,Avillius,Avillius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"nomen singulare, viri",sigilla impressa,nomen singulare; sigilla impressa; tituli fa...,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.50691 39.47598),True,41221.0,27923.0,1000.0,rural,None,NaN
3,LIST-4,EDCS-55701595,None,None,None,None,L(ucius) Av(illius),L Av,None,Lucius Avillius,Lucius Avillius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"praenomen et nomen, viri",sigilla impressa,praenomen et nomen; sigilla impressa; tituli...,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.50691 39.47598),True,41221.0,27923.0,1000.0,rural,None,NaN
4,LIST-5,EDCS-55701596,None,None,None,None,Camurius,Camurius,None,Camurius,Camurius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"nomen singulare, viri",sigilla impressa,nomen singulare; sigilla impressa; tituli fa...,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,[ ],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.50691 39.47598),True,41221.0,27923.0,1000.0,rural,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529846,LIST-529847,None,HD082771,None,207361,------] /[--- vixit an]nis / [--- Ger?]man/[us...,None,nis man ex arioV,vixit annis Germanus veteranus ex cohortis sag...,vixit annis Germanus veteranus ex cohortis sag...,None,] / 

In [ ]:
LIST.to_parquet("../data/large_data/LIST_contexts.parquet")

# Additional deduplication

In [3]:
LIST = gpd.read_parquet("../data/large_data/LIST_contexts.parquet")

In [4]:
edh_ids_list = LIST[LIST["EDH-ID"].notnull()]["EDH-ID"].tolist()
len(edh_ids_list)

81266

In [5]:
# records with valid & UNIQUE EDH-ID
edh_ids_unique = LIST[LIST["EDH-ID"].notnull()]["EDH-ID"].unique().tolist()
len(edh_ids_unique)

79899

In [7]:
# EDH-IDs appearing at least twice:
dup_ids = [id_tup[0] for id_tup in nltk.FreqDist(edh_ids_list).most_common() if id_tup[1] > 1]
len(dup_ids)

1136

In [8]:
# dataframe of duplicate rows (i.e. every second instance of the same id
LIST[LIST["EDH-ID"].notnull()].loc[LIST.duplicated(subset="EDH-ID"), :]

,LIST-ID,EDCS-ID,EDH-ID,trismegistos_uri,pleiades_id,transcription,inscription,clean_text_conservative,clean_text_interpretive_sentence,clean_text_interpretive_word,clean_text_interpretive_word_EDCS,diplomatic_text,province,place,inscr_type,status_notation,inscr_process,status,partner_link,last_update,letter_size,type_of_inscription,work_status,year_of_find,present_location,text_edition,support_objecttype,support_material,support_decoration,keywords_term,people,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,language,language_EDCS,raw_dating,not_after,not_before,Longitude,Latitude,geotemporal?,geometry,withinRE?,large_city_pop,medium_city_pop,small_city_pop,urban_context,urban_context_city,urban_context_pop_est
449357,LIST-449358,EDCS-28500267,HD064154,https://www.trismegistos.org/text/178167,599588,[---]S[---] // [---]T[---] // [---]SA[---] // ...,]T[,S T SA M G N I DE O K OR CVF,S T SA M G N I DE O K OR CVF,S T SA M G N I DE O K OR CVF,T,[ ]S[ ] / / [ ]T[ ] / / [ ]SA[ ] / / [ ]M[ ] /...,Achaia,Delos / Ortygia,None,None,None,None,http://db.edcs.eu/epigr/partner.php?s_language...,2011-11-18,17 cm,None,no image,NaN,None,\n S T SA M G N I DE O K OR CVF \n,None,NaN,1000.0,143.0,[ ],NULL,False,NaN,NaN,NaN,NULL,NULL,False,Achaia,False,Greece,False,Delos,False,Nótio Aigaío,False,Dhílos,False,NULL,False,L,None,None,NaN,NaN,25.2661,37.4036,False,POINT (25.26610 37.40360),False,58112.0,18364.0,2290.0,medium,Delos,18364.0
449358,LIST-449359,EDCS-28500268,HD064154,https://www.trismegistos.org/text/178167,599588,[---]S[---] // [---]T[---] // [---]SA[---] // ...,]SA[,S T SA M G N I DE O K OR CVF,S T SA M G N I DE O K OR CVF,S T SA M G N I DE O K OR CVF,SA,[ ]S[ ] / / [ ]T[ ] / / [ ]SA[ ] / / [ ]M[ ] /...,Achaia,Delos / Ortygia,None,None,None,None,http://db.edcs.eu/epigr/partner.php?s_language...,2011-11-18,17 cm,None,no image,NaN,None,\n S T SA M G N I DE O K OR CVF \n,None,NaN,1000.0,143.0,[ ],NULL,False,NaN,NaN,NaN,NULL,NULL,False,Achaia,False,Greece,False,Delos,False,Nótio Aigaío,False,Dhílos,False,NULL,False,L,None,None,NaN,NaN,25.2661,37.4036,False,POINT (25.26610 37.40360),False,58112.0,18364.0,2290.0,medium,Delos,18364.0
449359,LIST-449360,EDCS-28500269,HD064154,https://www.trismegistos.org/text/178167,599588,[---]S[---] // [---]T[---] // [---]SA[---] // ...,]M[,S T SA M G N I DE O K OR CVF,S T SA M G N I DE O K OR CVF,S T SA M G N I DE O K OR CVF,M,[ ]S[ ] / / [ ]T[ ] / / [ ]SA[ ] / / [ ]M[ ] /...,Achaia,Delos / Ortygia,None,None,None,None,http://db.edcs.eu/epigr/partner.php?s_language...,2011-11-18,17 cm,None,no image,NaN,None,\n S T SA M G N I DE O K OR CVF \n,None,NaN,1000.0,143.0,[ ],NULL,False,NaN,NaN,NaN,NULL,NULL,False,Achaia,False,Greece,False,Delos,False,Nótio Aigaío,False,Dhílos,False,NULL,False,L,None,None,NaN,NaN,25.2661,37.4036,False,POINT (25.26610 37.40360),False,58112.0,18364.0,2290.0,medium,Delos,18364.0
449360,LIST-449361,EDCS-28500271,HD064154,https://www.trismegistos.org/text/178167,599588,[---]S[---] // [---]T[---] // [---]SA[---] // ...,]N[,S T SA M G N I DE O K OR CVF,S T SA M G N I DE O K OR CVF,S T SA M G N I DE O K OR CVF,N,[ ]S[ ] / / [ ]T[ ] / / [ ]SA[ ] / / [ ]M[ ] /...,Achaia,Delos / Ortygia,None,None,None,None,http://db.edcs.eu/epigr/partner.php?s_language...,2011-11-18,17 cm,None,no image,NaN,None,\n S T SA M G N I DE O K OR CVF \n,None,NaN,1000.0,143.0,[ ],NULL,False,NaN,NaN,NaN,NULL,NULL,False,Achaia,False,Greece,False,Delos,False,Nótio Aigaío,False,Dhílos,False,NULL,False,L,None,None,NaN,NaN,25.2661,37.4036,False,POINT (25.26610 37.40360),False,58112.0,18364.0,2290.0,medium,Delos,18364.0
449361,LIST-449362,EDCS-28500272,HD064154,https://www.trismegi

In [9]:
# remove the duplicates
# (using pandas deduplicated() method)
LIST = LIST.loc[~(LIST.duplicated(subset="EDH-ID")) | (LIST["EDH-ID"].isnull())]
len(LIST)

528484

In [10]:
EDH_overlap_all = LIST[(LIST["EDH-ID"].notnull()) & (LIST["EDCS-ID"].notnull())]
len(EDH_overlap_all)

76583

In [11]:
len(LIST[(LIST["EDH-ID"].notnull()) & (LIST["EDCS-ID"].isnull())])

3316

In [16]:
len(LIST)

528484

In [12]:
len(LIST[(LIST["EDH-ID"].isnull()) & (LIST["EDCS-ID"].notnull())])

448585

In [13]:
contexts = ["large", "medium", "small", "rural"]
for context in contexts:
    print(context, (LIST["urban_context"]==context).sum())

large 156693
medium 130890
small 81022
rural 131425


# Saving the data

In [15]:
LIST.to_parquet("../data/large_data/LIST_contexts.parquet")